This notebook tries to work out what utc_time format is

In [3]:
import h5py
import xarray as xr
import glob
import matplotlib.pyplot as plt
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd
from shapely.ops import nearest_points
import numpy as np

In [4]:
def icesat1todataframe(track):
    """
    this function will import the .mat and arrange into a dataframe
    track = eg track
    """
    
    
    
    path = f'/Volumes/arc_01/horganhu/ICESAT/GLA12_633_TRACKS/{track}_processed.mat'
    df = pd.DataFrame({'x':[],
                       'pass_num':[],
                      })
    psys = []
    h = []
    time = []
    dh = []

    with h5py.File(path, 'r') as f:
        #add x location
        psx_l = list(f['antarctica'][track]['psx'])
        for p in psx_l:
            df_temp = pd.DataFrame({'x': list(f['antarctica'][track]['psx'][p][0]),
                                    'pass_num': [p]*len(list(f['antarctica'][track]['psx'][p])[0]) })
            df = df.append(df_temp , ignore_index=True )

        #add y location
        psy_l = list(f['antarctica'][track]['psy'])
        for p in psy_l:
            psys.extend( list(f['antarctica'][track]['psy'][p][0]) )

        #add height
        h_l = list(f['antarctica'][track]['elev_wgs84_retide'])
        for p in h_l:
            h.extend( list(f['antarctica'][track]['elev_wgs84_retide'][p][0]) )

        #add time
        t_l = list(f['antarctica'][track]['UTCTime'])
        for p in t_l:
            time.extend( list(f['antarctica'][track]['UTCTime'][p][0]) )

        #add delta_h (not sure what it is)
        dh_l = list(f['antarctica'][track]['delta_h'])
        for p in dh_l:
            dh.extend( list(f['antarctica'][track]['delta_h'][p][0]) )

    df['y'] = psys
    df['h'] = h
    df['time'] = time
    df['delta_h'] = dh

    da = df.query("x > -382064.5722209641 & x < -374781.1654740692 & y > -734075.0820404041 & y < -722764.4514729496")
    da.reset_index(drop=True,inplace=True)
    
    points = [Point(xy) for xy in zip(da.x,da.y)]
    gda = gpd.GeoDataFrame(da,geometry=points,crs=3031)
    
    return gda

### from load_matlab_icesat1_structures we see the tracks over the channel with  the most data
track number_of_datapoints
track0015 0
track1331 68
track0043 0
track0197 68
track1303 68
track0351 68
track0057 0
track0407 68
track0155 68
track0127 68
track0183 68
track0365 68
track0337 68
track0099 753
track0295 68
track0323 68
track0211 549
track0113 68
track0379 68
track0085 193
track0267 68
track0071 0
track0169 68
track1289 68
track0029 0
track1345 68
track0239 68
track1275 68
track0225 68
track0001 0
track0393 68
track0141 68
track0253 68
track0309 68
track1317 68
track0281 68

In [5]:
df = icesat1todataframe('track0099')

    % Note that ICESat i_UTCTime is seconds and microseconds referenced to noon on Jan 1st 2000
    % the matlab serial date number is days and fractional days since
    % 1-Jan-0000
`
disp('---------------')
disp('time conversion')
startt=(datenum('2000,01,01','yyyy,mm,dd')+0.5)*86400; % Noon Jan 1st, 2000, in seconds
sdatedays=(UTCTime+startt)/86400;
`

`datenum('2000,01,01','yyyy,mm,dd') = 730486` (run on matlab)

In [20]:
icesat_UTCtime_func = lambda t : pd.Timestamp.fromtimestamp(t)
df.time.apply(icesat_UTCtime_func)

0     2004-05-31 12:49:38.967118
1     2004-05-31 12:49:38.992118
2     2004-05-31 12:49:39.017118
3     2004-05-31 12:49:39.042118
4     2004-05-31 12:49:39.067118
                 ...            
748   2003-10-29 19:40:08.305512
749   2003-10-29 19:40:08.330512
750   2003-10-29 19:40:08.355512
751   2003-10-29 19:40:08.380512
752   2003-10-29 19:40:08.405512
Name: time, Length: 753, dtype: datetime64[ns]

In [42]:
icesat_UTCtime_func = lambda t : pd.Timestamp.utcfromtimestamp(t)+ (pd.Timestamp(2000,1,1,12) - pd.Timestamp(1970,1,1))
df.time.apply(icesat_UTCtime_func)

0     2004-05-31 00:49:38.967118
1     2004-05-31 00:49:38.992118
2     2004-05-31 00:49:39.017118
3     2004-05-31 00:49:39.042118
4     2004-05-31 00:49:39.067118
                 ...            
748   2003-10-29 07:40:08.305512
749   2003-10-29 07:40:08.330512
750   2003-10-29 07:40:08.355512
751   2003-10-29 07:40:08.380512
752   2003-10-29 07:40:08.405512
Name: time, Length: 753, dtype: datetime64[ns]

In [43]:
pd.Timedelta(30*365.25,'days') - (pd.Timestamp(2000,1,1,12) - pd.Timestamp(1970,1,1))

Timedelta('0 days 00:00:00')

ie the Timedelta and Timestamp are 

In [34]:
import datetime
np.datetime64(datetime.datetime(2000, 1, 1)) + df.time.to_numpy(dtype=np.timedelta64)

array(['2000-01-01T00:02:19.236578', '2000-01-01T00:02:19.236578',
       '2000-01-01T00:02:19.236579', '2000-01-01T00:02:19.236579',
       '2000-01-01T00:02:19.236579', '2000-01-01T00:02:19.236579',
       '2000-01-01T00:02:19.236579', '2000-01-01T00:02:19.236579',
       '2000-01-01T00:02:19.236579', '2000-01-01T00:02:19.236579',
       '2000-01-01T00:02:19.236579', '2000-01-01T00:02:19.236579',
       '2000-01-01T00:02:19.236579', '2000-01-01T00:02:19.236579',
       '2000-01-01T00:02:19.236579', '2000-01-01T00:02:19.236579',
       '2000-01-01T00:02:19.236579', '2000-01-01T00:02:19.236579',
       '2000-01-01T00:02:19.236579', '2000-01-01T00:02:19.236579',
       '2000-01-01T00:02:19.236579', '2000-01-01T00:02:19.236579',
       '2000-01-01T00:02:19.236579', '2000-01-01T00:02:19.236579',
       '2000-01-01T00:02:19.236579', '2000-01-01T00:02:19.236579',
       '2000-01-01T00:02:19.236579', '2000-01-01T00:02:19.236579',
       '2000-01-01T00:02:19.236579', '2000-01-01T00:02:19.2365

In [23]:
pd.Timedelta(30*365.25, 'days')

Timedelta('10957 days 12:00:00')